# 허깅페이스의 임베딩 모델
허깅페이스에는 여러 임베딩 모델이 존재합니다. 이 중 한글을 가장 잘 임베딩 할 수 있는 모델인 bge-m3 모델을 사용해 문장을 임베딩하고, 유사도를 구해봅시다.

In [2]:
# !pip install -U sentence-transformers
# !pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 9.3 MB/s eta 0:00:00


## BAAU/bge-m3의 스팩정보는 허깅페이스 홈페이지에서 확인이 가능합니다.

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\user\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

# bge-m3 기반 임베딩 수행

In [2]:
embedded_vector = model.encode("야 저기 차 온다")
embedded_vector.shape

(1024,)

In [3]:
import pandas as pd

data = [
    "내일 차타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=['text'])
df

,text
0,내일 차타고 놀러가자
1,지금 오는 버스는 어디서 오는 버스야?
2,차 한잔 하면서 이야기 하시죠
3,5차 공동구매! 오늘만 세일!
4,홍차 녹차 중에 어떤 차가 좋으세요?


In [4]:
def get_embedding(text) :
    # bge-m3 인코더를 이용해 문장에 대한 임베딩 벡터 얻어내기
    return list(model.encode(text))

df['embedding'] = df.apply(lambda row : get_embedding(row.text), axis = 1)
df

,text,embedding
0,내일 차타고 놀러가자,"[0.0054786936, 0.016985165, -0.020413598, 0.02..."
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388524, 0.024189435, -0.025403254, -0.0..."
2,차 한잔 하면서 이야기 하시죠,"[-0.00221578, 0.010684625, -0.039016668, 0.008..."
3,5차 공동구매! 오늘만 세일!,"[-0.030319404, -0.0011549796, -0.033044472, 0...."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.036115166, 0.0046489765, -0.021344062, -0.0..."


# 유사도 구하기
- 코사인 유사도

In [5]:
import numpy as np

def cos_sim(A, B) :
    return A @ B / (np.linalg.norm(A) * np.linalg.norm(B))

In [ ]:
def return_answer_candiadate(df, query) :
    """ 
        df : 문장과 임베딩 벡터가 들어있는 데이터 프레임
        query : 질의할 문장
    """

    # 질의에 대한 임베딩 벡터
    query_embedding = get_embedding(query)
    
    df['similarity'] = df.embedding.apply(
        lambda x : cos_sim(np.array(x), np.array(query_embedding)))
    
    # 입력한 문장(query)와 데이터 세트 내에 있는 문장으 ㅣ유사도를 내림차순으로 정렬
    result_df = df.sort_values("similarity", ascending=False, ignore_index = True)

    return result_df.head(3)

In [7]:
return_answer_candiadate(df, "야 저기 차 온다")

,text,embedding,similarity
0,내일 차타고 놀러가자,"[0.0054786936, 0.016985165, -0.020413598, 0.02...",0.720349
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388524, 0.024189435, -0.025403254, -0.0...",0.710821
2,차 한잔 하면서 이야기 하시죠,"[-0.00221578, 0.010684625, -0.039016668, 0.008...",0.664830
